In [ ]:
import requests

try:
  with open("in.dta", "x") as infile:
    request_in = requests.get("http://work.caltech.edu/data/in.dta")
    infile.write(request_in.text)
    print("Downloaded training data")
except FileExistsError as e:
  print("Training data already downloaded")

try:
  outfile = open("out.dta", "x")
  request_out = requests.get("http://work.caltech.edu/data/out.dta")
  outfile.write(request_out.text)
  print("Downloaded test data")
except FileExistsError as e:
  print("Test data already downloaded")

Downloaded training data
Downloaded test data


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

train_data = np.loadtxt("in.dta")
test_data = np.loadtxt("out.dta")

def transform(X):
  x_1 = X[:, 0]
  x_2 = X[:, 1]
  ones = np.ones_like(x_1)
  newX = np.column_stack([ones, x_1, x_2, x_1**2, x_2**2, x_1 * x_2, abs(x_1 - x_2), abs(x_1 + x_2)])
  return newX

def error(X, Y, w, k, N):
  err = 0
  z = transform(X)[:, :k + 1]

  for i in range(N):
    predict = np.sign(np.dot(w.T, z[i]))
    actual = Y[i]
    if predict != actual:
      err += 1

  return err / N

In [ ]:
trainN = 25

trainX, trainY = train_data[:trainN, :2], train_data[:trainN, 2]
valX, valY = train_data[trainN:, :2], train_data[trainN:, 2]
testX, testY = test_data[:, :2], test_data[:, 2]

for k in range(3, 8):
  trainX_transform = transform(trainX)[:, :k + 1]
  X_transpose = np.transpose(trainX_transform)
  X_transpose_X = np.matmul(X_transpose, trainX_transform)
  X_inverse = np.linalg.inv(X_transpose_X)
  X_dagger = np.matmul(X_inverse, X_transpose)
  w = np.matmul(X_dagger, trainY)

  print("k:", k)
  print("Validation Sample Error:", error(valX, valY, w, k, valX.shape[0]))
  print("Out Sample Error:", error(testX, testY, w, k, testX.shape[0]))
  print("______________________")

k: 3
Validation Sample Error: 0.3
Out Sample Error: 0.42
______________________
k: 4
Validation Sample Error: 0.5
Out Sample Error: 0.416
______________________
k: 5
Validation Sample Error: 0.2
Out Sample Error: 0.188
______________________
k: 6
Validation Sample Error: 0.0
Out Sample Error: 0.084
______________________
k: 7
Validation Sample Error: 0.1
Out Sample Error: 0.072
______________________


In [ ]:
trainX, trainY = valX, valY
valX, valY = train_data[:trainN, :2], train_data[:trainN, 2]
testX, testY = test_data[:, :2], test_data[:, 2]

trainN = 10

for k in range(3, 8):
  trainX_transform = transform(trainX)[:, :k + 1]
  X_transpose = np.transpose(trainX_transform)
  X_transpose_X = np.matmul(X_transpose, trainX_transform)
  X_inverse = np.linalg.inv(X_transpose_X)
  X_dagger = np.matmul(X_inverse, X_transpose)
  w = np.matmul(X_dagger, trainY)

  print("k:", k)
  print("Validation Sample Error:", error(valX, valY, w, k, valX.shape[0]))
  print("Out Sample Error:", error(testX, testY, w, k, testX.shape[0]))
  print("______________________")

k: 3
Validation Sample Error: 0.28
Out Sample Error: 0.396
______________________
k: 4
Validation Sample Error: 0.36
Out Sample Error: 0.388
______________________
k: 5
Validation Sample Error: 0.2
Out Sample Error: 0.284
______________________
k: 6
Validation Sample Error: 0.08
Out Sample Error: 0.192
______________________
k: 7
Validation Sample Error: 0.12
Out Sample Error: 0.196
______________________


Choosing k = 6 for both chosen models since these have the smallest validation error, we have

.084, .0192

In [ ]:
runs = 10000

ind_var = np.random.uniform(0, 1, (runs, 3))

for row in range(runs):
    ind_var[row, 2] = min(ind_var[row ,0],ind_var[row ,1])

print(f'e_1 = {np.mean(ind_var[:, 0])}')
print(f'e_2 = {np.mean(ind_var[:, 1])}')
print(f'min(e_1, e_2) = {np.mean(ind_var[:, 2])}')

e_1 = 0.5009784931948063
e_2 = 0.49919132413141626
min(e_1, e_2) = 0.3339825099140385


In [ ]:
import random
from cvxopt import matrix, solvers

MAX_ITER = 10000

def random_point():
    return [np.random.uniform(-1, 1), np.random.uniform(-1, 1)]

def find_y(x, m, b):
    pos = m * x[0] - x[1] + b
    return np.sign(pos)

def generate_points(N):
  while True:
    pt1 = random_point()
    pt2 = random_point()
    while pt1[0] == pt2[0]:
      pt2 = random_point()

    m = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0])
    b = pt1[1] - m * pt1[0]

    X = []
    Y = []

    for i in range(N):
      new_pt = random_point()
      y = find_y(new_pt, m, b)
      X.append(new_pt)
      Y.append(y)

    X = np.array(X)
    Y = np.array(Y)

    if not (np.all(Y == 1) or np.all(Y == -1)):
      break

  return X, Y, m, b

def pla(X, Y):
  N = X.shape[0]
  w = np.zeros(3)
  iterations = 0
  ones = np.ones((N, 1))
  X_aug = np.hstack((ones, X))

  while iterations < MAX_ITER:
    predictions = np.sign(np.dot(X_aug, w))
    misclassified = np.where(predictions != Y)[0]

    if misclassified.size == 0:
      break

    random_idx = random.choice(misclassified)
    w += Y[random_idx] * X_aug[random_idx]

    iterations += 1

  return w, iterations

def svm(X, Y):
  N, dim = X.shape

  # get into standard QP form
  P = np.zeros((dim + 1, dim + 1))
  P[:dim, :dim] = np.eye(dim)
  q = np.zeros(dim + 1)

  # constraints G x <= h
  G = np.zeros((N, dim + 1))
  h = np.zeros(N)

  for i in range(N):
    G[i, :dim] = -Y[i] * X[i]
    G[i, dim] = -Y[i] * 1
    h[i] = -1

  P = matrix(P)
  q = matrix(q)
  G = matrix(G)
  h = matrix(h)

  solvers.options['show_progress'] = False
  solution = solvers.qp(P, q, G, h)

  w = np.array(solution['x']).flatten()
  w_svm = w[:dim]
  b_svm = w[dim]

  # check for support vectors
  # if constraint is close to 0 with margin of error then support vector
  constraint = Y * (np.dot(X, w_svm) + b_svm) - 1
  support_vectors = np.where(constraint <= 1e-4)[0]
  num_sv = len(support_vectors)

  return w_svm, b_svm, num_sv

def measure_disagreement(m, b_line, w, b_svm=None, N=10000):
  testX = np.random.uniform(-1, 1, (N, 2))
  y_f = np.sign(m * testX[:, 0] - testX[:, 1] + b_line)

  if b_svm is not None:
    y_g = np.sign(np.dot(testX, w) + b_svm)
  else:
    ones = np.ones((N, 1))
    testX_tmp = np.hstack((ones, testX))
    y_g = np.sign(np.dot(testX_tmp, w))

  disagreement = np.mean(y_f != y_g)

  return disagreement

def experiment(N):
  runs = 1000
  svm_better_count = 0
  total_sv = 0

  for i in range(runs):
    X, Y, m, b_line = generate_points(N)

    w_pla, iterations = pla(X, Y)
    w_svm, b_svm, num_sv = svm(X, Y)

    total_sv += num_sv

    pla_disagreement = measure_disagreement(m, b_line, w_pla)
    svm_disagreement = measure_disagreement(m, b_line, w_svm, b_svm)

    if pla_disagreement > svm_disagreement:
      svm_better_count += 1

  svm_impr_performance = (svm_better_count / runs) * 100
  avg_sv = total_sv / runs

  print(f"SVM Performance Improvement: {svm_impr_performance}")
  print(f"Avg support vectors: {avg_sv}")

In [ ]:
experiment(10)

SVM Performance Improvement: 59.599999999999994
Avg support vectors: 2.853


In [ ]:
experiment(100)

SVM Performance Improvement: 64.2
Avg support vectors: 2.995
